# Description
this jupyter notebook converts the data provided by geosn available here https://www.geodaten.sachsen.de/downloadbereich-digitale-hoehenmodelle-4851.html into a height map.
The resulting height map can then be used to create a custom map for a game such as the farming simulator by giants.

## Input format
The input.zip is a zip file containing zip folders where each containing zip file contains a .xyz and a .csv file. The csv file contains metadata about the tile. The .xyz file contains height informations about each coordinate in the square.
Be a aware that the folder ./input is used as a temporary folder, which will be deleted if it already exists. Make sure no important data is located in this folder if you intend to run this jupyter notebook.
Depending on the size of the input data, a large amount of storage may be used. For the heightmap generation of a 8km by 8km square, roughly 6 GByte of RAM are being used by the jupyter notebook.

## Output
The data is converted into an image, which can further be resized and saved to disk. The saving is done by the Python Image Library, which supports a variety of formats.
The output is written to ./output.png.

# Extract input data

In [99]:
import shutil
shutil.rmtree("./input")

In [100]:
import zipfile
with zipfile.ZipFile("./input.zip", 'r') as zip_ref:
    zip_ref.extractall("./input/stage1")

In [101]:
import os;
for filename in os.listdir("./input/stage1"):
    if filename.endswith(".zip"):
        with zipfile.ZipFile(os.path.join("./input/stage1", filename)) as zip_ref:
            zip_ref.extractall("./input/stage2")

# process and read in data

In [102]:
import glob

xStart = 0;
yStart = 0;

xEnd = 0;
yEnd = 0;

tileXDim = 0;
tileYDim = 0;

# find xStart and yStart

tileDims = []
for name in glob.glob("./input/stage2/*.csv"):
    with open(name,"r") as file:
        csvreader = csv.reader(file)
        # read all the values of the csvreader
        values = list(csvreader)
        tileDims.append([values[1][0].split(";")[1].split(" "), name])
tileDims.sort()
print(tileDims)
xStart = int(tileDims[0][0][0])
yStart = int(tileDims[0][0][1])
xEnd = int(tileDims[-1][0][2])
yEnd = int(tileDims[-1][0][3])
tileXDim = int(tileDims[0][0][2]) - int(tileDims[0][0][0])
tileYDim = int(tileDims[0][0][3]) - int(tileDims[0][0][1])

xOffset = xStart * -1;
yOffset = yStart * -1;
print(xStart, yStart, xEnd, yEnd, tileXDim, tileYDim)
print(xEnd+xOffset, yEnd + yOffset)

[[['338000', '5666000', '340000', '5668000'], './input/stage2\\dgm1_33338_5666_2_sn_akt.csv'], [['338000', '5668000', '340000', '5670000'], './input/stage2\\dgm1_33338_5668_2_sn_akt.csv'], [['338000', '5670000', '340000', '5672000'], './input/stage2\\dgm1_33338_5670_2_sn_akt.csv'], [['338000', '5672000', '340000', '5674000'], './input/stage2\\dgm1_33338_5672_2_sn_akt.csv'], [['340000', '5666000', '342000', '5668000'], './input/stage2\\dgm1_33340_5666_2_sn_akt.csv'], [['340000', '5668000', '342000', '5670000'], './input/stage2\\dgm1_33340_5668_2_sn_akt.csv'], [['340000', '5670000', '342000', '5672000'], './input/stage2\\dgm1_33340_5670_2_sn_akt.csv'], [['340000', '5672000', '342000', '5674000'], './input/stage2\\dgm1_33340_5672_2_sn_akt.csv'], [['342000', '5666000', '344000', '5668000'], './input/stage2\\dgm1_33342_5666_2_sn_akt.csv'], [['342000', '5668000', '344000', '5670000'], './input/stage2\\dgm1_33342_5668_2_sn_akt.csv'], [['342000', '5670000', '344000', '5672000'], './input/stage

In [103]:
import numpy as np

# access via data[x][y]
data = np.zeros(dtype="float32", shape=(xEnd+xOffset, yEnd+yOffset))

print(data)
print(data.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(8000, 8000)


In [104]:
# file = "./input/stage2/dom1_33338_5666_2_sn.xyz"
import csv
# read file line by line and split by whitespace
i = 0;
for file in glob.glob("./input/stage2/*.xyz"):
    with open(file, 'r') as f:
        for line in f.readlines():
            x,y,z = map(lambda el : float(el),line.split(" "))
            x = round(x) + xOffset;
            y = round(y) + yOffset;
            data[x,y] = z;
    i+=1;
    print("completed #{0}".format(i))


completed #1
completed #2
completed #3
completed #4
completed #5
completed #6
completed #7
completed #8
completed #9
completed #10
completed #11
completed #12
completed #13
completed #14
completed #15
completed #16


In [105]:
print(data)

[[196.39 196.33 196.44 ... 160.7  160.69 160.65]
 [196.36 196.35 196.34 ... 160.62 160.61 160.64]
 [196.39 196.37 196.42 ... 160.58 160.6  160.59]
 ...
 [143.13 143.15 143.27 ... 161.69 161.58 161.42]
 [143.4  143.69 143.75 ... 161.9  161.71 161.61]
 [143.97 144.11 144.19 ... 161.92 161.87 161.77]]


# Output data

In [109]:
norm_data = (255*(data - np.min(data))/np.ptp(data)).astype(int) ;
print(norm_data)
print(np.max(norm_data))
from PIL import Image
# convert values to 0 - 255 int8 format
formatted = (norm_data).astype('uint8')
img = Image.fromarray(formatted).rotate(90)
img = img.resize(size=(4097,4097)) # uncomment to resize the output
img.save("./output.tiff", format="TIFF")
print("Heightscale: {0}".format(np.max(data)-np.min(data)))

[[158 158 158 ...  74  74  74]
 [158 158 158 ...  74  74  74]
 [158 158 158 ...  74  74  74]
 ...
 [ 33  33  33 ...  76  76  76]
 [ 33  34  34 ...  77  77  76]
 [ 35  35  35 ...  77  77  77]]
255
Heightscale: 108.47000122070312
